In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pymannkendall import original_test

from dataflow import df_train, df_test, df_geography, df_product

# 1. Kiểm chứng các giả thuyết

In [2]:
# Kết nối tự nhiên giữa các bảng
df = df_train.merge(df_geography, on='Zip', how='left').merge(df_product, on='ProductID', how='left')

In [3]:
df.head(10)

,ProductID,Date,Zip,Units,Revenue,COGS,City,State,Region,District,Category,Segment,Product
0,1228,2013-07-31,78773,12,19648.44,12309.747660,"Austin, TX",TX,Central,District #23,Urban,Convenience,Pirum UC-30
1,781,2014-03-12,90501,16,20351.52,13497.128064,"Torrance, CA",CA,West,District #37,Rural,Productivity,Natura RP-69
2,2090,2013-11-29,44460,26,111367.62,91488.499830,"Salem, OH",OH,East,District #14,Urban,Convenience,Currus UC-25
3,1116,2018-08-29,30122,12,36280.44,21967.806420,"Lithia Springs, GA",GA,East,District #09,Rural,Select,Pirum RS-04
4,993,2013-04-27,75789,14,55557.18,48645.866808,"Troup, TX",TX,Central,District #24,Urban,Convenience,Natura UC-56
5,715,2014-12-29,27707,12,23806.44,21909.066732,"Durham, NC",NC,East,District #08,Rural,Productivity,Natura RP-03
6,808,2016-11-25,77041,13,53636.31,31693.695579,"Houston, TX",TX,Central,District #23,Rural,Select,Natura RS-12
7,764,2012-01-27,27522,40,146134.80,135496.186560,"Creedmoor, NC",NC,East,District #08,Rural,Productivity,Natura RP-52
8,433,2016-04-17,90250,21,240719.85,137451.034350,"Hawthorne, CA",CA,West,District #37,Urban,Moderation,Maximus UM-38
9,1244,2017-01-06,33178,17,98510.58,71016.277122,"Miami, FL",FL,East,District #10,Mix,Productivity,Quibus MP-12


## 1.1. Giả thuyết 1

### Kiểm định ANOVA: Doanh thu trung bình có khác biệt giữa các khu vực (Region)?
- $\mathcal{H}_0$ (Giả thuyết không): Không có sự khác biệt đáng kể về doanh thu trung bình giữa các khu vực.
- $\mathcal{H}_1$ (Giả thuyết đối): Ít nhất một khu vực có doanh thu trung bình khác biệt đáng kể.


In [4]:
# Nhóm doanh thu theo Region
regions = df['Region'].unique()
revenue_by_region = [df[df['Region'] == region]['Revenue'] for region in regions]

# Thực hiện kiểm định ANOVA
anova_result = stats.f_oneway(*revenue_by_region)

print(f"ANOVA test result for Revenue across Regions: p-value = {anova_result.pvalue}")

# Kiểm định kết luận
alpha = 0.05
if anova_result.pvalue < alpha:
    print("Có sự khác biệt đáng kể giữa các khu vực.")
else:
    print("Không có sự khác biệt đáng kể giữa các khu vực.")


ANOVA test result for Revenue across Regions: p-value = 0.0
Có sự khác biệt đáng kể giữa các khu vực.


## 1.2. Giả thuyết 2

### Kiểm định ANOVA: Số lượng sản phẩm bán ra có khác nhau giữa các danh mục (Category)?
- $\mathcal{H}_0$: Không có sự khác biệt đáng kể về số lượng bán giữa các danh mục sản phẩm.
- $\mathcal{H}_1$: Ít nhất một danh mục có số lượng bán khác biệt đáng kể.

In [5]:
# Nhóm số lượng sản phẩm bán theo Category
categories = df['Category'].unique()
units_by_category = [df[df['Category'] == category]['Units'] for category in categories]

# Thực hiện kiểm định ANOVA
anova_result_units = stats.f_oneway(*units_by_category)

print(f"ANOVA test result for Units across Categories: p-value = {anova_result_units.pvalue}")

if anova_result_units.pvalue < alpha:
    print("Có sự khác biệt đáng kể về số lượng bán giữa các danh mục sản phẩm.")
else:
    print("Không có sự khác biệt đáng kể về số lượng bán giữa các danh mục sản phẩm.")

ANOVA test result for Units across Categories: p-value = 0.0
Có sự khác biệt đáng kể về số lượng bán giữa các danh mục sản phẩm.


## 1.3. Giả thuyết 3

###  T-Test: Doanh thu trung bình trước và sau năm 2016 có khác biệt đáng kể?
- $\mathcal{H}_0$: Không có sự khác biệt đáng kể về doanh thu trung bình trước và sau năm 2016.
- $\mathcal{H}_1$: Có sự khác biệt đáng kể về doanh thu trung bình giữa hai giai đoạn.

In [6]:
# Chia dữ liệu thành hai nhóm: trước 2016 và từ 2016 trở đi
df['Year'] = df['Date'].dt.year
before_2016 = df[df['Year'] < 2016]['Revenue']
after_2016 = df[df['Year'] >= 2016]['Revenue']

# Thực hiện kiểm định T-test
ttest_result = stats.ttest_ind(before_2016, after_2016, equal_var=False)  # Welch’s T-test

print(f"T-Test result for Revenue before and after 2016: p-value = {ttest_result.pvalue}")

if ttest_result.pvalue < alpha:
    print("Có sự khác biệt đáng kể về doanh thu trung bình trước và sau năm 2016.")
else:
    print("Không có sự khác biệt đáng kể về doanh thu trung bình trước và sau năm 2016.")

T-Test result for Revenue before and after 2016: p-value = 0.0
Có sự khác biệt đáng kể về doanh thu trung bình trước và sau năm 2016.


## 1.4. Giả thuyết 4

### Kiểm định tương quan Pearson: Doanh thu và số lượng bán có tương quan dương?
- $\mathcal{H}_0$: Không có mối tương quan đáng kể giữa doanh thu và số lượng bán.
- $\mathcal{H}_1$: Có tương quan giữa doanh thu và số lượng bán.

In [7]:
# Tính hệ số tương quan Pearson
corr_coef, p_value = stats.pearsonr(df['Units'], df['Revenue'])

print(f"Pearson Correlation giữa Units và Revenue: r = {corr_coef}, p-value = {p_value}")

if p_value < alpha:
    print("Có mối tương quan đáng kể giữa số lượng bán và doanh thu.")
else:
    print("Không có mối tương quan đáng kể giữa số lượng bán và doanh thu.")

Pearson Correlation giữa Units và Revenue: r = 0.32574882047269005, p-value = 0.0
Có mối tương quan đáng kể giữa số lượng bán và doanh thu.


## 1.5. Giả thuyết 5

### Kiểm định ANOVA: COGS có khác nhau giữa các phân khúc sản phẩm?
- $\mathcal{H}_0$: Không có sự khác biệt đáng kể về COGS giữa các phân khúc (Segment).
- $\mathcal{H}_1$: Có ít nhất một phân khúc có COGS khác biệt đáng kể

In [8]:
# Nhóm COGS theo Segment
segments = df['Segment'].unique()
cogs_by_segment = [df[df['Segment'] == segment]['COGS'] for segment in segments]

# Thực hiện kiểm định ANOVA
anova_result_cogs = stats.f_oneway(*cogs_by_segment)

print(f"ANOVA test result for COGS across Segments: p-value = {anova_result_cogs.pvalue}")

if anova_result_cogs.pvalue < alpha:
    print("Có sự khác biệt đáng kể về COGS giữa các phân khúc sản phẩm.")
else:
    print("Không có sự khác biệt đáng kể về COGS giữa các phân khúc sản phẩm.")

ANOVA test result for COGS across Segments: p-value = 0.0
Có sự khác biệt đáng kể về COGS giữa các phân khúc sản phẩm.


## 2. Tạo đặc trưng mới và thực hiện kiểm chứng các giả thuyết

In [9]:
# Tạo cột Lợi nhuận (Profit)
df['Profit'] = df['Revenue'] - df['COGS']

# Tạo cột Doanh thu trung bình mỗi đơn vị sản phẩm
df['Revenue_per_unit'] = df['Revenue'] / df['Units']

# Tạo cột Lợi nhuận trung bình mỗi đơn vị sản phẩm
df['Profit_per_unit'] = df['Profit'] / df['Units']

# Xử lý các giá trị NaN (do phép chia có thể tạo ra NaN khi Units = 0)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

# Tạo cột Weekday (1-7, với Monday=0)
df['Weekday'] = df['Date'].dt.weekday
df['Is_Weekend'] = df['Weekday'].apply(lambda x: 1 if x >= 5 else 0)  # 1 = Cuối tuần, 0 = Ngày thường

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901561 entries, 0 to 901560
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   ProductID         901561 non-null  int64         
 1   Date              901561 non-null  datetime64[ns]
 2   Zip               901561 non-null  int64         
 3   Units             901561 non-null  int64         
 4   Revenue           901561 non-null  float64       
 5   COGS              901561 non-null  float64       
 6   City              901561 non-null  object        
 7   State             901561 non-null  object        
 8   Region            901561 non-null  object        
 9   District          901561 non-null  object        
 10  Category          901561 non-null  object        
 11  Segment           901561 non-null  object        
 12  Product           901561 non-null  object        
 13  Year              901561 non-null  int32         
 14  Prof

## 2.1. Giả thuyết 6

### Kiểm định ANOVA: Lợi nhuận trung bình có khác biệt giữa các khu vực?
- $\mathcal{H}_0$: Không có sự khác biệt về lợi nhuận trung bình giữa các khu vực.
- $\mathcal{H}_1$: Có ít nhất một khu vực có lợi nhuận trung bình khác biệt đáng kể.

In [11]:
regions = df['Region'].unique()
profit_by_region = [df[df['Region'] == region]['Profit'] for region in regions]

anova_result_profit = stats.f_oneway(*profit_by_region)

print(f"ANOVA test result for Profit across Regions: p-value = {anova_result_profit.pvalue}")

if anova_result_profit.pvalue < 0.05:
    print("Có sự khác biệt đáng kể về lợi nhuận trung bình giữa các khu vực.")
else:
    print("Không có sự khác biệt đáng kể về lợi nhuận giữa các khu vực.")

ANOVA test result for Profit across Regions: p-value = 2.499182978740946e-302
Có sự khác biệt đáng kể về lợi nhuận trung bình giữa các khu vực.


## 2.2. Giả thuyết 7

### Kiểm định xu hướng Mann-Kendall: Doanh thu trung bình có xu hướng tăng qua các năm không?
- $\mathcal{H}_0$: Không có xu hướng tăng hoặc giảm rõ ràng của doanh thu theo thời gian.
- $\mathcal{H}_1$: Doanh thu có xu hướng tăng hoặc giảm theo thời gian.

In [12]:
# Sắp xếp dữ liệu theo ngày
df_sorted = df.groupby('Date').agg({'Revenue': 'sum'}).reset_index()

# Thực hiện kiểm định xu hướng Mann-Kendall
trend_result = original_test(df_sorted['Revenue'])

print(f"Mann-Kendall trend test result: {trend_result.trend}, p-value = {trend_result.p}")

Mann-Kendall trend test result: decreasing, p-value = 0.0


## 2.3. Giả thuyết 8

### Kiểm định ANOVA: Giá bán trung bình có khác biệt giữa các danh mục sản phẩm?
- $\mathcal{H}_0$: Không có sự khác biệt đáng kể về giá bán trung bình giữa các danh mục sản phẩm.
- $\mathcal{H}_1$: Có ít nhất một danh mục có giá bán trung bình khác biệt đáng kể.

In [13]:
categories = df['Category'].unique()
revenue_per_unit_by_category = [df[df['Category'] == category]['Revenue_per_unit'] for category in categories]

anova_result_rpu = stats.f_oneway(*revenue_per_unit_by_category)

print(f"ANOVA test result for Revenue_per_unit across Categories: p-value = {anova_result_rpu.pvalue}")

if anova_result_rpu.pvalue < 0.05:
    print("Có sự khác biệt đáng kể về giá bán trung bình giữa các danh mục sản phẩm.")
else:
    print("Không có sự khác biệt đáng kể về giá bán trung bình giữa các danh mục sản phẩm.")

ANOVA test result for Revenue_per_unit across Categories: p-value = 0.0
Có sự khác biệt đáng kể về giá bán trung bình giữa các danh mục sản phẩm.


## 2.4. Giả thuyết 9

### Kiểm định Kruskal-Wallis: Lợi nhuận trung bình mỗi đơn vị có khác nhau giữa các khu vực?
- $\mathcal{H}_0$: Không có sự khác biệt đáng kể về lợi nhuận trung bình mỗi đơn vị (Profit_per_unit) giữa các khu vực.
- $\mathcal{H}_1$: Có ít nhất một khu vực có lợi nhuận trung bình mỗi đơn vị khác biệt đáng kể.

In [14]:
kruskal_result = stats.kruskal(*[df[df['Region'] == region]['Profit_per_unit'] for region in df['Region'].unique()])

print(f"Kruskal-Wallis test result for Profit_per_unit across Regions: p-value = {kruskal_result.pvalue}")

if kruskal_result.pvalue < 0.05:
    print("Có sự khác biệt đáng kể về lợi nhuận trung bình mỗi đơn vị giữa các khu vực.")
else:
    print("Không có sự khác biệt đáng kể về lợi nhuận trung bình mỗi đơn vị giữa các khu vực.")

Kruskal-Wallis test result for Profit_per_unit across Regions: p-value = 2.3181445544012354e-199
Có sự khác biệt đáng kể về lợi nhuận trung bình mỗi đơn vị giữa các khu vực.


## 2.5. Giả thuyết 10

### T-Test: Doanh thu vào cuối tuần và ngày thường có khác biệt không?
- $\mathcal{H}_0$: Không có sự khác biệt về doanh thu trung bình giữa ngày cuối tuần và ngày thường.
- $\mathcal{H}_1$: Doanh thu trung bình có sự khác biệt giữa ngày cuối tuần và ngày thường.

In [15]:
weekend_revenue = df[df['Is_Weekend'] == 1]['Revenue']
weekday_revenue = df[df['Is_Weekend'] == 0]['Revenue']

t_test_weekend = stats.ttest_ind(weekend_revenue, weekday_revenue, equal_var=False)

print(f"T-Test result for Revenue on weekends vs weekdays: p-value = {t_test_weekend.pvalue}")

if t_test_weekend.pvalue < 0.05:
    print("Có sự khác biệt đáng kể về doanh thu trung bình giữa ngày cuối tuần và ngày thường.")
else:
    print("Không có sự khác biệt đáng kể về doanh thu giữa ngày cuối tuần và ngày thường.")


T-Test result for Revenue on weekends vs weekdays: p-value = 3.7298184822298854e-17
Có sự khác biệt đáng kể về doanh thu trung bình giữa ngày cuối tuần và ngày thường.


# 3. Tính toán các độ đo kinh tế học và kiểm chứng giả thuyết

In [16]:
# Tạo chỉ số Tỷ suất lợi nhuận gộp (Gross Margin Rate - GMR)
df['Gross_Margin_Rate'] = (df['Profit'] / df['Revenue']) * 100

# Tạo chỉ số Tỷ suất lợi nhuận trên chi phí (Return on Cost - ROC)
df['Return_on_Cost'] = (df['Profit'] / df['COGS']) * 100

# Tạo chỉ số Tỷ lệ bán hàng theo từng sản phẩm (Sales Share per Product - SSPP)
total_units_sold = df['Units'].sum()
df['Sales_Share_per_Product'] = df['Units'] / total_units_sold

# Tạo chỉ số Tỷ lệ doanh thu theo từng khu vực (Revenue Share by Region - RSR)
df['Revenue_Share_by_Region'] = df.groupby('Region')['Revenue'].transform(lambda x: x / x.sum())

# Tạo chỉ số Hệ số biến động doanh thu theo tuần (Revenue Variability - RV)
df['Week'] = df['Date'].dt.isocalendar().week
df_weekly = df.groupby(['Year', 'Week']).agg({'Revenue': ['mean', 'std']}).reset_index()
df_weekly.columns = ['Year', 'Week', 'Revenue_Mean', 'Revenue_Std']
df_weekly['Revenue_Variability'] = df_weekly['Revenue_Std'] / df_weekly['Revenue_Mean']

# Gộp trở lại với `df`
df = df.merge(df_weekly[['Year', 'Week', 'Revenue_Variability']], on=['Year', 'Week'], how='left')

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901561 entries, 0 to 901560
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ProductID                901561 non-null  int64         
 1   Date                     901561 non-null  datetime64[ns]
 2   Zip                      901561 non-null  int64         
 3   Units                    901561 non-null  int64         
 4   Revenue                  901561 non-null  float64       
 5   COGS                     901561 non-null  float64       
 6   City                     901561 non-null  object        
 7   State                    901561 non-null  object        
 8   Region                   901561 non-null  object        
 9   District                 901561 non-null  object        
 10  Category                 901561 non-null  object        
 11  Segment                  901561 non-null  object        
 12  Product         

## 3.1. Giả thuyết 11

### Kiểm chứng ANOVA: Tỷ suất lợi nhuận gộp có khác nhau giữa các khu vực?
- $\mathcal{H}_0$: Không có sự khác biệt về tỷ suất lợi nhuận gộp giữa các khu vực.
- $\mathcal{H}_1$: Có ít nhất một khu vực có tỷ suất lợi nhuận gộp khác biệt đáng kể.

In [18]:
regions = df['Region'].unique()
gmr_by_region = [df[df['Region'] == region]['Gross_Margin_Rate'].dropna() for region in regions]

anova_result_gmr = stats.f_oneway(*gmr_by_region)

print(f"ANOVA test result for Gross Margin Rate across Regions: p-value = {anova_result_gmr.pvalue}")

if anova_result_gmr.pvalue < 0.05:
    print("Có sự khác biệt đáng kể về tỷ suất lợi nhuận gộp giữa các khu vực.")
else:
    print("Không có sự khác biệt đáng kể về tỷ suất lợi nhuận gộp giữa các khu vực.")

ANOVA test result for Gross Margin Rate across Regions: p-value = 3.8835903696967476e-27
Có sự khác biệt đáng kể về tỷ suất lợi nhuận gộp giữa các khu vực.


## 3.2. Giả thuyết 12

### Kiểm chứng Kruskal-Wallis: Tỷ suất lợi nhuận trên chi phí có khác nhau giữa các danh mục sản phẩm?
- $\mathcal{H}_0$: Không có sự khác biệt về tỷ suất lợi nhuận trên chi phí giữa các danh mục sản phẩm.
- $\mathcal{H}_1$: Có ít nhất một danh mục có tỷ suất lợi nhuận trên chi phí khác biệt đáng kể.

In [19]:
categories = df['Category'].unique()
roc_by_category = [df[df['Category'] == category]['Return_on_Cost'].dropna() for category in categories]

kruskal_result_roc = stats.kruskal(*roc_by_category)

print(f"Kruskal-Wallis test for Return on Cost across Categories: p-value = {kruskal_result_roc.pvalue}")

if kruskal_result_roc.pvalue < 0.05:
    print("Có sự khác biệt đáng kể về tỷ suất lợi nhuận trên chi phí giữa các danh mục sản phẩm.")
else:
    print("Không có sự khác biệt đáng kể về tỷ suất lợi nhuận trên chi phí giữa các danh mục sản phẩm.")

Kruskal-Wallis test for Return on Cost across Categories: p-value = 0.0
Có sự khác biệt đáng kể về tỷ suất lợi nhuận trên chi phí giữa các danh mục sản phẩm.


## 3.3. Giả thuyết 13

### Kiểm chứng Chi-square: Các sản phẩm có đóng góp khác nhau vào tổng số lượng bán?
- $\mathcal{H}_0$: Không có sự khác biệt đáng kể về tỷ lệ bán hàng giữa các sản phẩm.
- $\mathcal{H}_1$: Có sự khác biệt đáng kể về tỷ lệ bán hàng giữa các sản phẩm.

In [20]:
from scipy.stats import chisquare

# Lấy số lượng bán của từng sản phẩm
product_sales = df.groupby('Product')['Units'].sum()

# Kiểm định Chi-square
chi2_result = chisquare(product_sales)

print(f"Chi-square test result for Sales Share per Product: p-value = {chi2_result.pvalue}")

if chi2_result.pvalue < 0.05:
    print("Có sự khác biệt đáng kể về tỷ lệ bán hàng giữa các sản phẩm.")
else:
    print("Không có sự khác biệt đáng kể về tỷ lệ bán hàng giữa các sản phẩm.")

Chi-square test result for Sales Share per Product: p-value = 0.0
Có sự khác biệt đáng kể về tỷ lệ bán hàng giữa các sản phẩm.


## 3.4. Giả thuyết 14

### Kiểm chứng tương quan Pearson: Biến động doanh thu giữa các tuần có ảnh hưởng đến lợi nhuận?
- $\mathcal{H}_0$: Không có tương quan giữa biến động doanh thu và lợi nhuận.
- $\mathcal{H}_1$: Có mối tương quan giữa biến động doanh thu và lợi nhuận.

In [21]:
corr_coef, p_value = stats.pearsonr(df['Revenue_Variability'], df['Profit'])

print(f"Pearson Correlation between Revenue Variability and Profit: r = {corr_coef}, p-value = {p_value}")

if p_value < 0.05:
    print("Có mối tương quan đáng kể giữa biến động doanh thu và lợi nhuận.")
else:
    print("Không có mối tương quan đáng kể giữa biến động doanh thu và lợi nhuận.")

Pearson Correlation between Revenue Variability and Profit: r = -0.0601227418376245, p-value = 0.0
Có mối tương quan đáng kể giữa biến động doanh thu và lợi nhuận.


## 3.5. Giả thuyết 15

### Kiểm chứng ANOVA/Kruskal-Wallis: Doanh thu có bị ảnh hưởng bởi mùa không?
- $\mathcal{H}_0$: Không có sự khác biệt đáng kể về doanh thu giữa các mùa.
- $\mathcal{H}_1$: Có ít nhất một mùa có doanh thu khác biệt đáng kể.

In [22]:
# Xác định mùa dựa trên tháng
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['Season'] = df['Date'].dt.month.apply(get_season)

In [23]:
from pandas.tseries.holiday import USFederalHolidayCalendar

# Lấy danh sách ngày lễ của Mỹ
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=df['Date'].min(), end=df['Date'].max())

# Tạo cột xác định ngày có phải là ngày lễ hay không
df['Is_Holiday'] = df['Date'].isin(holidays).astype(int)

In [24]:
df_variability_by_region = df.groupby('Region')['Revenue_Variability'].mean().reset_index()

In [25]:
seasons = df['Season'].unique()
revenue_by_season = [df[df['Season'] == season]['Revenue'] for season in seasons]

anova_result_season = stats.f_oneway(*revenue_by_season)

print(f"ANOVA test for Revenue across Seasons: p-value = {anova_result_season.pvalue}")

if anova_result_season.pvalue < 0.05:
    print("Có sự khác biệt đáng kể về doanh thu giữa các mùa.")
else:
    print("Không có sự khác biệt đáng kể về doanh thu giữa các mùa.")

ANOVA test for Revenue across Seasons: p-value = 0.0
Có sự khác biệt đáng kể về doanh thu giữa các mùa.


## 3.6. Giả thuyết 16

### Kiểm chứng Chi-square Test / ANOVA: Tỷ suất lợi nhuận theo năm có thay đổi không?
- $\mathcal{H}_0$: Không có sự khác biệt đáng kể về tỷ suất lợi nhuận giữa các năm.
- $\mathcal{H}_1$: Có ít nhất một năm có tỷ suất lợi nhuận khác biệt đáng kể.

In [26]:
years = df['Year'].unique()
profit_rate_by_year = [df[df['Year'] == year]['Gross_Margin_Rate'] for year in years]

anova_result_profit_year = stats.f_oneway(*profit_rate_by_year)

print(f"ANOVA test for Gross Margin Rate across Years: p-value = {anova_result_profit_year.pvalue}")

if anova_result_profit_year.pvalue < 0.05:
    print("Có sự khác biệt đáng kể về tỷ suất lợi nhuận giữa các năm.")
else:
    print("Không có sự khác biệt đáng kể về tỷ suất lợi nhuận giữa các năm.")

ANOVA test for Gross Margin Rate across Years: p-value = 2.4320826252348236e-235
Có sự khác biệt đáng kể về tỷ suất lợi nhuận giữa các năm.


## 3.7. Giả thuyết 17

### Kiểm chứng Kruskal-Wallis: Hệ số biến động doanh thu có khác nhau giữa các khu vực?
- $\mathcal{H}_0$: Không có sự khác biệt đáng kể về hệ số biến động doanh thu giữa các khu vực.
- $\mathcal{H}_1$: Có ít nhất một khu vực có hệ số biến động doanh thu khác biệt đáng kể.

In [27]:
regions = df['Region'].unique()
variability_by_region = [df[df['Region'] == region]['Revenue_Variability'] for region in regions]

kruskal_result_variability = stats.kruskal(*variability_by_region)

print(f"Kruskal-Wallis test for Revenue Variability across Regions: p-value = {kruskal_result_variability.pvalue}")

if kruskal_result_variability.pvalue < 0.05:
    print("Có sự khác biệt đáng kể về biến động doanh thu giữa các khu vực.")
else:
    print("Không có sự khác biệt đáng kể về biến động doanh thu giữa các khu vực.")

Kruskal-Wallis test for Revenue Variability across Regions: p-value = 0.0
Có sự khác biệt đáng kể về biến động doanh thu giữa các khu vực.


## 3.8. Giả thuyết 18

### Kiểm chứng T-Test: Doanh thu vào các ngày nghỉ lễ có cao hơn ngày thường không?
- $\mathcal{H}_0$: Không có sự khác biệt đáng kể về doanh thu giữa ngày lễ và ngày thường.
- $\mathcal{H}_1$: Có sự khác biệt đáng kể về doanh thu giữa ngày lễ và ngày thường.

In [28]:
holiday_revenue = df[df['Is_Holiday'] == 1]['Revenue']
non_holiday_revenue = df[df['Is_Holiday'] == 0]['Revenue']

t_test_holiday = stats.ttest_ind(holiday_revenue, non_holiday_revenue, equal_var=False)

print(f"T-Test for Revenue on Holidays vs Non-Holidays: p-value = {t_test_holiday.pvalue}")

if t_test_holiday.pvalue < 0.05:
    print("Có sự khác biệt đáng kể về doanh thu giữa ngày lễ và ngày thường.")
else:
    print("Không có sự khác biệt đáng kể về doanh thu giữa ngày lễ và ngày thường.")

T-Test for Revenue on Holidays vs Non-Holidays: p-value = 0.008387078738983929
Có sự khác biệt đáng kể về doanh thu giữa ngày lễ và ngày thường.
